In [5]:
import ibm_db
from pandas import DataFrame
import pandas as pd

In [6]:
import ibm_db_dbi as db

In [7]:
conn = db.connect("DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=sfz85754;PWD=j0-9fgh89tfj30b8;", "", "")

In [36]:
df = pd.read_sql("SELECT * from ADVISOR_DATA", conn)

In [37]:
# preprocess the database
database = df.set_index('name').T.to_dict('dict')

In [38]:
for key, value in database.items():
    value['language'] = [x.rstrip() for x in value['language'].split(',')]
    value['service'] = [x.rstrip() for x in value['service'].split(',')]

In [39]:
# mock query 
query = {'Smith White': {'language': ['Japanese', 'Hindi', 'Turkish'],
  'service': ['Consumer checking and savings', 'General investing', 'Home Equity'],
  'location': 'FL'},
        'Jordan Black': {'language': ['English', 'German', 'Italian'],
  'service': ['Business checking and savings', 'Merchant', 'Refinancing'],
  'location': 'SC'}}

In [40]:
# preprocess the query
query_map = {'Consumer checking and savings':'Consumer','Auto loans':'Consumer',\
       'Credit cards':'Consumer','Rewards':'Consumer',\
       'Retirement planning':'Investment','IRAs and 401(k)s':'Investment',\
       'General investing':'Investment','College planning':'Investment',\
       'Mortgage financing':'Home Loan','Refinancing':'Home Loan',\
       'Home Equity':'Home Loan',\
       'Business checking and savings':'Business','Lending':'Business',\
       'Payroll':'Business','Merchant':'Business','Financing':'Business'}       
# map the services into categories
for name,value in query.items():
    #print(value)
    result = value.pop("service") 
    res = []
    for ser in result:
        res.append(query_map[ser])
    value['service'] = res

In [41]:
query

{'Smith White': {'language': ['Japanese', 'Hindi', 'Turkish'],
  'location': 'FL',
  'service': ['Consumer', 'Investment', 'Home Loan']},
 'Jordan Black': {'language': ['English', 'German', 'Italian'],
  'location': 'SC',
  'service': ['Business', 'Business', 'Home Loan']}}

In [42]:
import numpy as np
import pandas as pd
import sys
import math
from collections import Counter

from pandas.io import sql

'''
input: user preference as txt file with features in order: 
language, 
service type, 
location
database: bank assistants as dict of dict of lists/int/string

'''



def sort_ranked_list(prefered_list):
    return(sorted(prefered_list, key = lambda x: x[1], reverse = True)[:3])

def select_bank_assistant(name):
    database[name]['availability'] = 0
    return

def finished_assistant(name):
    database[name]['availability'] = 1

def matching_algorithm(query,threshold):
    # set up a threshold for banking assistants to get into the list
    matched = []
    for name in database:
        if not database[name]['availability']:
            continue
        else:
            counterA = set(query['language'])
            counterB = set(database[name]['language'])
            cos_sim_lang = len(counterA.intersection(counterB))/len(counterA)
            counterA = set(query['service'])
            counterB = set(database[name]['service'])
            cos_sim_ser =len(counterA.intersection(counterB))/len(counterA)
            counterA = set(query['location'])
            counterB = set(database[name]['location'])
            cos_sim_loc =len(counterA.intersection(counterB))/len(counterA)
            score = 0.5 * cos_sim_lang + 0.4 * cos_sim_ser + 0.1 * cos_sim_loc
            if score > threshold:
                bank_assistant = name
                matched.append((bank_assistant,score))
    return matched
    

In [43]:
threshold = 0.5
query_result = dict()
for key,value in query.items(): 
    matched = matching_algorithm(value,threshold)
    ranked = sort_ranked_list(matched)
    teller_info = {key:[(x[0],database[x[0]]) for x in ranked]}
    query_result.update(teller_info)
    #print(ranked)

In [44]:
query_result['Smith White']

[('Lashandra Ritter',
  {'language': ['English', 'Spanish', 'Hindi'],
   'service': ['Investment', 'Home Loan', 'Consumer'],
   'location': 'FL',
   'availability': 1}),
 ('Brandis Givens',
  {'language': ['English', 'Hindi', 'Spanish'],
   'service': ['Home Loan', 'Consumer', 'Investment'],
   'location': 'VA',
   'availability': 1}),
 ('Shekinah Wyatt',
  {'language': ['English', 'Portugese', 'Japanese'],
   'service': ['Consumer', 'Investment', 'Home Loan'],
   'location': 'AZ',
   'availability': 1})]